### playnomore
- http://playnomore.co.kr/
- scrapy에서 fake-useragent 사용
- scrapy를 실행할때 아규먼트를 설정해서 실행
- pipelines에서 데이터 베이스로 데이터를 저장

In [2]:
import scrapy
import requests
from scrapy.http import TextResponse

#### 1. 프로젝트 생성

In [3]:
!rm -rf playnomore
!scrapy startproject playnomore

New Scrapy project 'playnomore', using template directory '/home/ubuntu/.pyenv/versions/3.6.9/envs/python3/lib/python3.6/site-packages/scrapy/templates/project', created in:
    /home/ubuntu/python3/notebook/playnomore

You can start your first spider with:
    cd playnomore
    scrapy genspider example example.com


#### 2. items.py
- title, price, img, link

In [44]:
import os

In [45]:
os.getcwd()

'/home/ubuntu/python3/notebook'

In [21]:
os.chdir('./python3/notebook')

FileNotFoundError: [Errno 2] No such file or directory: './python3/notebook'

In [4]:
%%writefile playnomore/playnomore/items.py
import scrapy

class PlaynomoreItem(scrapy.Item):
    title = scrapy.Field()
    price = scrapy.Field()
    img = scrapy.Field()
    link = scrapy.Field()

Overwriting playnomore/playnomore/items.py


#### 3. xpath 확인
- 링크
- 링크 -> 상세페이지(제목, 이미지URL, 가격)
- fake_useragent 설치
    - pip install fake_useragent

In [5]:
!pip install fake_useragent

You should consider upgrading via the '/home/ubuntu/.pyenv/versions/3.6.9/envs/python3/bin/python3.6 -m pip install --upgrade pip' command.


In [6]:
# 네트워크 > 헤더 > user agent값을 직접 복사해와서 사용 가능
from fake_useragent import UserAgent
url = "http://playnomore.co.kr/category/bag/24/"
headers = { "User-Agent": UserAgent().chrome }
# headers = { "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36" }
req = requests.get(url, headers=headers)
response = TextResponse(req.url, body=req.text, encoding="utf-8") 
response

<200 http://playnomore.co.kr/category/bag/24/>

In [7]:
#링크
links = response.xpath(
    '//*[@id="contents"]/div[2]/div/ul/li/div/a/@href'
)

In [8]:
links[0]

<Selector xpath='//*[@id="contents"]/div[2]/div/ul/li/div/a/@href' data='/product/detail.html?product_no=573&c...'>

In [10]:
# 문자열만 추출
links = response.xpath(
    '//*[@id="contents"]/div[2]/div/ul/li/div/a/@href'
).extract()

In [11]:
links[0]

'/product/detail.html?product_no=573&cate_no=24&display_group=1'

In [ ]:
# 도메인 추가

In [12]:
links = response.xpath(
    '//*[@id="contents"]/div[2]/div/ul/li/div/a/@href'
).extract()
links = list(map(response.urljoin, links))  # iterabel한 데이터 하나하나에 함수를 적용

In [35]:
links

['http://playnomore.co.kr/product/detail.html?product_no=573&cate_no=24&display_group=1',
 'http://playnomore.co.kr/product/detail.html?product_no=572&cate_no=24&display_group=1',
 'http://playnomore.co.kr/product/detail.html?product_no=550&cate_no=24&display_group=1',
 'http://playnomore.co.kr/product/detail.html?product_no=532&cate_no=24&display_group=1',
 'http://playnomore.co.kr/product/detail.html?product_no=507&cate_no=24&display_group=1',
 'http://playnomore.co.kr/product/detail.html?product_no=493&cate_no=24&display_group=1',
 'http://playnomore.co.kr/product/detail.html?product_no=545&cate_no=24&display_group=1',
 'http://playnomore.co.kr/product/detail.html?product_no=562&cate_no=24&display_group=1',
 'http://playnomore.co.kr/product/detail.html?product_no=563&cate_no=24&display_group=1',
 'http://playnomore.co.kr/product/detail.html?product_no=564&cate_no=24&display_group=1',
 'http://playnomore.co.kr/product/detail.html?product_no=565&cate_no=24&display_group=1',
 'http://p

In [14]:
# 상세페이지 : 제목, 가격, 이미지URL
url = links[0]
headers = { "User-Agent": UserAgent().chrome }
req = requests.get(url, headers=headers)
response = TextResponse(req.url, body=req.text, encoding="utf-8") 
response

<200 http://playnomore.co.kr/product/sale-20-micro-baguette-yellow-180/573/?cate_no=24&display_group=1>

In [23]:
title1 = response.xpath(
        '//*[@id="contents"]/div[1]/div[1]/div[2]/div[1]/font/text()'
    ).extract()
title2 = response.xpath(
        '//*[@id="contents"]/div[1]/div[1]/div[2]/div[1]/text()'
    ).extract()
title = "".join(title1) + "".join(title2)
price = response.xpath(
        '//*[@id="contents"]/div[1]/div[1]/div[2]/div[2]/text()'
    ).extract()[0]
img = "http:" + response.xpath(
        '//*[@id="contents"]/div[1]/div[1]/div[1]/div[1]/img/@src'
    ).extract()[0] 
title, price, img


('[SALE 20%] MICRO BAGUETTE yellow ',
 '$ 144',
 'http://playnomore.co.kr/web/product/big/20200407/626de3d4081df6e1dff057623e9e4877.jpg')

#### 4. spider.py
- scrapy-fake-useragent 설치
    - pip install scrapy-fake-useragent

In [25]:
!pip list | grep fake

fake-useragent        0.1.11      
scrapy-fake-useragent 1.2.0       
You should consider upgrading via the '/home/ubuntu/.pyenv/versions/3.6.9/envs/python3/bin/python3.6 -m pip install --upgrade pip' command.


In [36]:
%%writefile playnomore/playnomore/spiders/spider.py
import scrapy
from playnomore.items import PlaynomoreItem

class PlaynomoreSpider(scrapy.Spider):
    name = "Playnomore"
    custom_settings = {
        'DOWNLOADER_MIDDLEWARES': {
            'scrapy.downloadermiddlewares.useragent.UserAgentMiddleware': None,
            'scrapy_fake_useragent.middleware.RandomUserAgentMiddleware': 400,
        }
    }
    
    def start_requests(self):
        url = "http://playnomore.co.kr/category/bag/24/"
        yield scrapy.Request(url, callback=self.parse)
        
    def parse(self, response):
        links = response.xpath('//*[@id="contents"]/div[2]/div/ul/li/div/a/@href').extract()
        links = list(map(response.urljoin, links))
        for link in links:
            yield scrapy.Request(link, callback=self.page_parse)
    
    def page_parse(self, response):
        item = PlaynomoreItem()
        title1 = response.xpath('//*[@id="contents"]/div[1]/div[1]/div[2]/div[1]/font/text()').extract()
        title2 = response.xpath('//*[@id="contents"]/div[1]/div[1]/div[2]/div[1]/text()').extract()
        item["title"] = "".join(title1) + "".join(title2)
        item["price"] = response.xpath('//*[@id="contents"]/div[1]/div[1]/div[2]/div[2]/text()').extract()[0]
        item["img"] = "http:" + response.xpath('//*[@id="contents"]/div[1]/div[1]/div[1]/div[1]/img/@src').extract()[0]
        item["link"] = response.url
        yield item
            
        
    

Overwriting playnomore/playnomore/spiders/spider.py


In [29]:
%%writefile run.sh
cd playnomore
scrapy crawl Playnomore -o playnomore.csv


Overwriting run.sh


In [ ]:
# 실행권한 부여
!chmod +x run.sh

In [37]:
!./run.sh

2020-07-03 01:34:34 [scrapy.utils.log] INFO: Scrapy 2.2.0 started (bot: playnomore)
2020-07-03 01:34:34 [scrapy.utils.log] INFO: Versions: lxml 4.5.1.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.6.9 (default, Apr 15 2020, 12:56:52) - [GCC 7.5.0], pyOpenSSL 19.1.0 (OpenSSL 1.1.1g  21 Apr 2020), cryptography 2.9.2, Platform Linux-5.3.0-1023-aws-x86_64-with-debian-buster-sid
2020-07-03 01:34:34 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2020-07-03 01:34:34 [scrapy.crawler] INFO: Overridden settings:
{'BOT_NAME': 'playnomore',
 'NEWSPIDER_MODULE': 'playnomore.spiders',
 'ROBOTSTXT_OBEY': True,
 'SPIDER_MODULES': ['playnomore.spiders']}
2020-07-03 01:34:35 [scrapy.extensions.telnet] INFO: Telnet Password: 4eadb1e9515c911e
2020-07-03 01:34:35 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUs

In [38]:
import pandas as pd
df = pd.read_csv("playnomore/playnomore.csv")
df.tail(1)

,img,link,price,title
13,http://playnomore.co.kr/web/product/big/202003...,http://playnomore.co.kr/product/new-open-event...,$ 162,[New Open Event 10%] RAINBOW HOBO green


#### 5. argument 설정

In [49]:
%%writefile playnomore/playnomore/spiders/spider.py
import scrapy
from playnomore.items import PlaynomoreItem

class PlaynomoreSpider(scrapy.Spider):
    name = "Playnomore"
    custom_settings = {
        'DOWNLOADER_MIDDLEWARES': {
            'scrapy.downloadermiddlewares.useragent.UserAgentMiddleware': None,
            'scrapy_fake_useragent.middleware.RandomUserAgentMiddleware': 400,
        }
    }
    
    # 오버라이딩
    def __init__(self, category1="bag", category2=24, **kwargs):
        self.start_url = "http://playnomore.co.kr/category/{}/{}/".format(category1, category2)
        # 기존의 생성자 함수 살려주기
        super().__init__(**kwargs)
    
    def start_requests(self):
        url = self.start_url
        yield scrapy.Request(url, callback=self.parse)
        
    def parse(self, response):
        links = response.xpath('//*[@id="contents"]/div[2]/div/ul/li/div/a/@href').extract()
        links = list(map(response.urljoin, links))
        for link in links:
            yield scrapy.Request(link, callback=self.page_parse)
    
    def page_parse(self, response):
        item = PlaynomoreItem()
        title1 = response.xpath('//*[@id="contents"]/div[1]/div[1]/div[2]/div[1]/font/text()').extract()
        title2 = response.xpath('//*[@id="contents"]/div[1]/div[1]/div[2]/div[1]/text()').extract()
        item["title"] = "".join(title1) + "".join(title2)
        item["price"] = response.xpath('//*[@id="contents"]/div[1]/div[1]/div[2]/div[2]/text()').extract()[0]
        item["img"] = "http:" + response.xpath('//*[@id="contents"]/div[1]/div[1]/div[1]/div[1]/img/@src').extract()[0]
        item["link"] = response.url
        yield item

Overwriting playnomore/playnomore/spiders/spider.py


In [51]:
%%writefile run.sh
cd playnomore
scrapy crawl Playnomore -o playnomore2.csv -a category1=shoes -a category2=25

Overwriting run.sh


In [52]:
!./run.sh

2020-07-03 01:48:38 [scrapy.utils.log] INFO: Scrapy 2.2.0 started (bot: playnomore)
2020-07-03 01:48:38 [scrapy.utils.log] INFO: Versions: lxml 4.5.1.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.6.9 (default, Apr 15 2020, 12:56:52) - [GCC 7.5.0], pyOpenSSL 19.1.0 (OpenSSL 1.1.1g  21 Apr 2020), cryptography 2.9.2, Platform Linux-5.3.0-1023-aws-x86_64-with-debian-buster-sid
2020-07-03 01:48:38 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2020-07-03 01:48:38 [scrapy.crawler] INFO: Overridden settings:
{'BOT_NAME': 'playnomore',
 'NEWSPIDER_MODULE': 'playnomore.spiders',
 'ROBOTSTXT_OBEY': True,
 'SPIDER_MODULES': ['playnomore.spiders']}
2020-07-03 01:48:38 [scrapy.extensions.telnet] INFO: Telnet Password: abc3ba0a3372cb9c
2020-07-03 01:48:38 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUs

In [53]:
import pandas as pd
df = pd.read_csv("playnomore/playnomore2.csv")
df.tail(1)

,img,link,price,title
14,http://playnomore.co.kr/web/product/big/201702...,http://playnomore.co.kr/product/sold-out-winky...,$ 414,[SOLD OUT] WINKYGIRL COLOR BLOCKS (5cm) multi


#### 6. Mongodb에 저장
- pymongo를 piplines.py에 적용

In [55]:
!pip list | grep pymongo

pymongo               2.8.1       
You should consider upgrading via the '/home/ubuntu/.pyenv/versions/3.6.9/envs/python3/bin/python3.6 -m pip install --upgrade pip' command.


In [54]:
import pymongo

In [57]:
client = pymongo.MongoClient('mongodb://54.180.179.245:27017/')
client

MongoClient('54.180.179.245', 27017)

In [58]:
db = client.playnomore
collection = db.shoes
collection

Collection(Database(MongoClient('54.180.179.245', 27017), 'playnomore'), 'shoes')

In [59]:
data = {"title":"신발"}
collection.insert(data)

ObjectId('5efe91400bf77133c2406b4d')

##### Mongodb 모듈 파일 생성

In [60]:
%%writefile playnomore/playnomore/mongodb.py
import pymongo

client = pymongo.MongoClient('mongodb://54.180.179.245:27017/')
db = client.playnomore
collection = db.shoes

Writing playnomore/playnomore/mongodb.py


In [62]:
!cat writefile playnomore/playnomore/pipelines.py

cat: writefile: No such file or directory
# Define your item pipelines here
#
# Don't forget to add your pipeline to the ITEM_PIPELINES setting
# See: https://docs.scrapy.org/en/latest/topics/item-pipeline.html


# useful for handling different item types with a single interface
from itemadapter import ItemAdapter


class PlaynomorePipeline:
    def process_item(self, item, spider):
        return item


In [71]:
%%writefile playnomore/playnomore/pipelines.py
from .mongodb import collection

class PlaynomorePipeline(object):
    
    def process_item(self, item, spider):
        
        data = { "title": item["title"], 
                 "price": item["price"],
                 "img": item["img"], 
                 "link": item["link"],
               }
        
        collection.insert(data)
        
        return item

Overwriting playnomore/playnomore/pipelines.py


In [64]:
!echo "ITEM_PIPELINES = {" >> playnomore/playnomore/settings.py

In [65]:
!echo "   'playnomore.pipelines.PlaynomorePipeline': 300," >> playnomore/playnomore/settings.py

In [66]:
!echo "}" >> playnomore/playnomore/settings.py

In [72]:
!tail -n 5 playnomore/playnomore/settings.py

#HTTPCACHE_IGNORE_HTTP_CODES = []
#HTTPCACHE_STORAGE = 'scrapy.extensions.httpcache.FilesystemCacheStorage'
ITEM_PIPELINES = {
   'playnomore.pipelines.PlaynomorePipeline': 300,
}


In [73]:
!cat run.sh

cd playnomore
scrapy crawl Playnomore -o playnomore2.csv -a category1=shoes -a category2=25


In [74]:
!./run.sh

2020-07-03 02:26:28 [scrapy.utils.log] INFO: Scrapy 2.2.0 started (bot: playnomore)
2020-07-03 02:26:28 [scrapy.utils.log] INFO: Versions: lxml 4.5.1.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.6.9 (default, Apr 15 2020, 12:56:52) - [GCC 7.5.0], pyOpenSSL 19.1.0 (OpenSSL 1.1.1g  21 Apr 2020), cryptography 2.9.2, Platform Linux-5.3.0-1023-aws-x86_64-with-debian-buster-sid
2020-07-03 02:26:28 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2020-07-03 02:26:28 [scrapy.crawler] INFO: Overridden settings:
{'BOT_NAME': 'playnomore',
 'NEWSPIDER_MODULE': 'playnomore.spiders',
 'ROBOTSTXT_OBEY': True,
 'SPIDER_MODULES': ['playnomore.spiders']}
2020-07-03 02:26:28 [scrapy.extensions.telnet] INFO: Telnet Password: 9046cbc8b4c40c6d
2020-07-03 02:26:28 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUs